In [1]:
from google.colab import auth
import google.auth
from google.cloud import bigquery
import plotly.express as px
import pandas as pd

auth.authenticate_user()

project_id = 'tactile-471816'
client = bigquery.Client(project=project_id)

In [53]:
#@title Player Distribution by Launch Phase
%%bigquery installs --project {project_id}

SELECT
  case when install_date is null then null ELSE DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', install_date)) end as install_date,
  CASE
    WHEN DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', install_date)) < '2022-04-01' THEN 'Soft Launch - CA'
    WHEN DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', install_date)) >= '2022-06-06' THEN 'Official Launch - US'
    ELSE 'Unknown'
  END as launch_phase,
  case
    when lower(platform) = 'google_android' then 'android'
    when lower(platform) = 'apple_ios' then 'ios'
    else lower(platform)
  end as platform,
  LEFT(channel_country,2) AS country,
  SPLIT(channel_country, '-')[OFFSET(1)] AS channel_type,
  case when gender is null then "unknown" else gender end as gender,
  CASE
    WHEN age IS NULL OR age = '' OR age = 'unknown' THEN 'unknown'
    WHEN age = '60+' THEN '60+'
    WHEN SAFE_CAST(age AS INT64) IS NULL THEN 'unknown'
    WHEN SAFE_CAST(age AS INT64) <=30 THEN '<=30'
    WHEN SAFE_CAST(age AS INT64) BETWEEN 31 AND 40 THEN '31-40'
    WHEN SAFE_CAST(age AS INT64) BETWEEN 41 AND 50 THEN '41-50'
    WHEN SAFE_CAST(age AS INT64) BETWEEN 51 AND 60 THEN '51-60'
    ELSE 'unknown'
  END AS age_group,
  count(DISTINCT user_id) as num_player,
  count(*) as num_device
FROM `tactile-471816.data_analyst_test_local.users`
WHERE DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', install_date)) >= '2020-01-01'
  AND DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', install_date)) <= CURRENT_DATE()
GROUP BY 1, 2, 3, 4, 5, 6, 7
ORDER BY 1

Query is running:   0%|          |

Downloading:   0%|          |

In [54]:
installs.head(5)

,install_date,launch_phase,platform,country,channel_type,gender,age_group,num_player,num_device
0,2022-02-01,Soft Launch - CA,android,CA,marketing,unknown,<=30,1,1
1,2022-02-01,Soft Launch - CA,android,CA,marketing,unknown,31-40,2,2
2,2022-02-01,Soft Launch - CA,android,CA,marketing,female,41-50,11,11
3,2022-02-01,Soft Launch - CA,android,CA,marketing,female,51-60,5,5
4,2022-02-01,Soft Launch - CA,android,CA,marketing,female,<=30,2,2


We can also look at these weather stations in an interactive world map, using [Plotly's geographical scatterplot functionality](https://plotly.com/python/scatter-plots-on-maps/).

In [60]:
# Remove rows where install_date is null
df_clean = installs.dropna(subset=['install_date'])

# Convert to datetime with error handling
df_clean['install_date'] = pd.to_datetime(df_clean['install_date'], format='%Y-%m-%d', errors='coerce')

# Remove invalid dates
df_clean = df_clean.dropna(subset=['install_date'])

# Filter to reasonable range
df_clean = df_clean[(df_clean['install_date'] >= '2020-01-01') &
                    (df_clean['install_date'] <= pd.Timestamp.now())]

# Group by install_date and sum num_player
timeseries_data = df_clean.groupby('install_date')['num_player'].sum().reset_index()

# Assign country/color categories
def assign_region(date):
    month = date.month
    if month < 4:   # Jan, Feb, Mar
        return "Canada"
    elif month >= 6:  # June onward
        return "US"
    else:
        return "Other"

timeseries_data['region'] = timeseries_data['install_date'].apply(assign_region)

# Create bar chart
fig = px.bar(
    timeseries_data,
    x='install_date',
    y='num_player',
    color='region',
    color_discrete_map={'Canada': 'red', 'US': 'blue'},
    title='Number of Players by Install Date'
)

fig.update_layout(
    xaxis_title='Install Date',
    yaxis_title='Number of Players',
    xaxis_tickangle=-45
)

fig.show()


In [63]:
# Extract day of week and week number
df_clean = df_clean[df_clean['country'] == 'US']

df_clean['day_of_week'] = df_clean['install_date'].dt.day_name()
df_clean['week_start'] = df_clean['install_date'] - pd.to_timedelta(df_clean['install_date'].dt.weekday, unit='d')
df_clean['week_label'] = df_clean['week_start'].dt.strftime('%Y-%m-%d')

# Exclude specific week
df_clean = df_clean[(df_clean['week_label'] != '2022-06-27') ]

# Ensure proper weekday order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_clean['day_of_week'] = pd.Categorical(df_clean['day_of_week'], categories=weekday_order, ordered=True)

# Aggregate installs per day of week per week
weekly_summary = df_clean.groupby(['week_label', 'day_of_week'])['num_player'].sum().reset_index()

# Plot multiple lines, one per week
fig = px.line(
    weekly_summary,
    x='day_of_week',
    y='num_player',
    color='week_label',
    markers=True,
    title='US Player Installs by Day of Week'
)

fig.update_layout(
    xaxis_title='Day of Week',
    yaxis_title='Number of Players',
    legend_title='Week Starting',
)

fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Helper function to group and sort data
def prepare_data(column, sort_by_value=True, custom_order=None):
    data = df_clean.groupby(column)['num_player'].sum().reset_index()
    if custom_order:
        data[column] = pd.Categorical(data[column], categories=custom_order, ordered=True)
        data = data.sort_values(column)
    elif sort_by_value:
        data = data.sort_values('num_player', ascending=False)  # descending order
    else:
        data = data.sort_values(column)
    return data

# Prepare datasets (no Country now)
datasets = {
    'Platform': prepare_data('platform'),
    'Gender': prepare_data('gender'),
    'Channel Type': prepare_data('channel_type'),
    'Age Group': prepare_data(
        'age_group',
        sort_by_value=False,
        custom_order=['<=30', '31-40', '41-50', '51-60', '60+', 'unknown']
    ),
}

# Create subplot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Platform Distribution',
        'Gender Distribution',
        'Channel Type Distribution',
        'Age Group Distribution'
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Add traces
data_list = list(datasets.values())

fig.add_trace(go.Bar(x=data_list[0]['platform'], y=data_list[0]['num_player']), row=1, col=1)
fig.add_trace(go.Bar(x=data_list[1]['gender'], y=data_list[1]['num_player']), row=1, col=2)
fig.add_trace(go.Bar(x=data_list[2]['channel_type'], y=data_list[2]['num_player']), row=2, col=1)
fig.add_trace(go.Bar(x=data_list[3]['age_group'], y=data_list[3]['num_player']), row=2, col=2)

fig.update_layout(
    height=700,
    showlegend=False,
    title_text="US Player Distribution Analysis"
)

fig.show()


In [ ]:
%%bigquery level --project {project_id}
SELECT
  date,
  COUNT(DISTINCT user_id) as daily_active_users,
  SUM(levels_played) as total_levels_played,
  SUM(levels_completed) as total_levels_completed
FROM `tactile-471816.data_analyst_test_local.activity`
WHERE date IS NOT NULL and date >='2022-06-06'
GROUP BY date
ORDER BY date

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df = level

# Convert date to datetime if it's not already
df['date'] = pd.to_datetime(df['date'])

# Calculate per-user averages
df['avg_levels_played_per_user'] = df['total_levels_played'] / df['daily_active_users']
df['avg_levels_completed_per_user'] = df['total_levels_completed'] / df['daily_active_users']
df['completion_rate'] = df['total_levels_completed'] / df['total_levels_played']*100


# Create subplots (3 rows × 1 column)
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        'Levels Played per User',
        'Levels Completed per User',
        'Completion Rate'
    ),
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": False}],
           ]
)

# Add line traces
fig.add_trace(go.Scatter(
    x=df['date'], y=df['avg_levels_played_per_user'],
    mode='lines+markers', name='Avg Levels Played Per User',
    line=dict(color="red")
), row=1, col=1)



# Avg Levels Completed per User
fig.add_trace(go.Scatter(
    x=df['date'], y=df['avg_levels_completed_per_user'],
    mode='lines+markers', name='Avg Levels Completed/User',
    line=dict(color="green")
), row=2, col=1)

# Levels Completion Rate
fig.add_trace(go.Scatter(
    x=df['date'], y=df['completion_rate'],
    mode='lines+markers', name='Levels Completion Rate',
    line=dict(color="black")
), row=3, col=1)

# Update layout
fig.update_layout(
    height=900,
    showlegend=True,  # ✅ turned legend on to distinguish avg vs median
    title_text="Daily Player Engagement Metrics"
)

fig.update_xaxes(title_text="Date")

fig.show()


In [ ]:
%%bigquery daily_retention --project {project_id}

WITH user_daily_activity AS (
  SELECT
    user_id,
    date,
    LAG(date) OVER (PARTITION BY user_id ORDER BY date) as prev_date,
  FROM `tactile-471816.data_analyst_test_local.activity`
  WHERE date IS NOT NULL AND  date >='2022-06-06' AND user_id IS NOT NULL
),
daily_returns AS (
  SELECT
    date,
    COUNT(DISTINCT user_id) as total_active_users,
    COUNT(DISTINCT CASE WHEN DATE_DIFF(date, prev_date, DAY) = 1 THEN user_id END) as returned_next_day,
  FROM user_daily_activity
  GROUP BY date
),
return_rates AS (
  SELECT
    date,
    total_active_users,
    returned_next_day,
    ROUND(returned_next_day * 100.0 / NULLIF(total_active_users, 0), 2) as daily_return_rate_pct,
  FROM daily_returns
)
SELECT * FROM return_rates
ORDER BY date

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df = daily_retention
df['date'] = pd.to_datetime(df['date'])

# Single plot for daily return rate
fig = px.line(df, x='date', y='daily_return_rate_pct',
              title='Daily Return Rate - Players Coming Back Each Day',
              markers=True)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Daily Return Rate (%)",
    yaxis=dict(range=[0, 100])
)

fig.show()

# Add summary stats
print(f"Average daily return rate: {df['daily_return_rate_pct'].mean():.1f}%")


Average daily return rate: 57.4%


In [ ]:
%%bigquery progression --project {project_id}

WITH user_max_levels AS (
  SELECT
    user_id,
    MIN(date) as first_active_date,
    MAX(max_level_completed) as highest_level_reached
  FROM `tactile-471816.data_analyst_test_local.activity`
  WHERE date IS NOT NULL
    AND date >= '2022-06-06'
    AND user_id IS NOT NULL
    AND max_level_completed IS NOT NULL
  GROUP BY user_id
),
milestone_levels AS (
  -- Generate milestone levels from 100 to 5000 in increments of 100
  SELECT level
  FROM UNNEST(GENERATE_ARRAY(100, 5000, 100)) as level
),
user_milestone_achievements AS (
  SELECT
    uml.user_id,
    uml.first_active_date,
    uml.highest_level_reached,
    ml.level as milestone_level,
    CASE WHEN uml.highest_level_reached >= ml.level THEN uml.first_active_date ELSE NULL END as milestone_achieved
  FROM user_max_levels uml
  CROSS JOIN milestone_levels ml
)
SELECT
  milestone_level as level,
  COUNT(DISTINCT user_id) as total_users_attempted,
  COUNT(milestone_achieved) as users_reached,
  ROUND(COUNT(milestone_achieved) * 100.0 / COUNT(DISTINCT user_id), 2) as completion_rate_pct
FROM user_milestone_achievements
GROUP BY milestone_level
HAVING COUNT(milestone_achieved) > 0
ORDER BY milestone_level

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
progression.head(100)

,level,total_users_attempted,users_reached,completion_rate_pct
0,100,19552,4697,24.02
1,200,19552,2869,14.67
2,300,19552,1918,9.81
3,400,19552,1273,6.51
4,500,19552,911,4.66
5,600,19552,662,3.39
6,700,19552,487,2.49
7,800,19552,378,1.93
8,900,19552,315,1.61
9,1000,19552,268,1.37


In [ ]:
%%bigquery two_week_retention --project {project_id}

WITH user_installs AS (
  SELECT
    u.user_id,
    REGEXP_EXTRACT(u.user_id, r'_(\d+)') as numeric_user_id,
    DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', u.install_date)) as install_date
  FROM `tactile-471816.data_analyst_test_local.users` u
  WHERE u.install_date IS NOT NULL
    AND DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', u.install_date)) >= '2020-01-01' -- Filter out invalid dates
    AND DATE(SAFE.PARSE_TIMESTAMP('%Y-%m-%dT%H:%M:%E*SZ', u.install_date)) <= CURRENT_DATE()
),
user_activity AS (
  SELECT
    CAST(a.user_id AS STRING) as numeric_user_id,
    a.date as activity_date
  FROM `tactile-471816.data_analyst_test_local.activity` a
  WHERE a.date IS NOT NULL AND a.user_id IS NOT NULL
),
two_week_retention AS (
  SELECT
    ui.user_id,
    ui.install_date,
    CASE
      WHEN ui.install_date < '2022-06-01' THEN 'Soft Launch'
      ELSE 'Official Launch'
    END as launch_phase,
    CASE
      WHEN MAX(CASE
        WHEN DATE_DIFF(ua.activity_date, ui.install_date, DAY) BETWEEN 14 AND 20
        THEN 1 ELSE 0
      END) = 1 THEN 1
      ELSE 0
    END as active_week_2
  FROM user_installs ui
  LEFT JOIN user_activity ua ON ui.numeric_user_id = ua.numeric_user_id
  WHERE ui.install_date <= DATE_SUB(CURRENT_DATE(), INTERVAL 21 DAY)
  GROUP BY ui.user_id, ui.install_date, launch_phase
)
SELECT
  launch_phase,
  COUNT(*) as total_installed_users,
  SUM(active_week_2) as users_active_week_2,
  ROUND(SUM(active_week_2) * 100.0 / COUNT(*), 2) as two_week_retention_pct
FROM two_week_retention
GROUP BY launch_phase
ORDER BY launch_phase



Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
two_week_retention.head()

,launch_phase,total_installed_users,users_active_week_2,two_week_retention_pct
0,Official Launch,21391,1833,8.57
1,Soft Launch,7675,1808,23.56


In [ ]:
%%bigquery monetisation --project {project_id}

-- Step 1: Daily revenue by type and country/launch phase
WITH daily_revenue AS (
  SELECT
    eventDate AS revenue_date,
    revenue_type,
    SUM(transaction_value) AS total_revenue,
    CASE
      WHEN EXTRACT(MONTH FROM eventDate) < 4 THEN 'Canada - Soft Launch'
      WHEN EXTRACT(MONTH FROM eventDate) >= 6 THEN 'US - Official Launch'
      ELSE 'Other'
    END AS launch_phase,
    ARRAY_AGG(DISTINCT user_id) AS spender_list
  FROM `tactile-471816.data_analyst_test_local.revenues`
  WHERE eventDate IS NOT NULL
    AND transaction_value IS NOT NULL
  GROUP BY revenue_date, revenue_type, launch_phase
),

-- Step 2: Pivot IAP vs Ads revenue and calculate total paying users per day & launch phase
pivot_revenue AS (
  SELECT
    revenue_date,
    launch_phase,
    SUM(CASE WHEN revenue_type = 'iap' THEN total_revenue ELSE 0 END) AS iap_revenue,
    SUM(CASE WHEN revenue_type = 'ad' THEN total_revenue ELSE 0 END) AS ad_revenue,
    ARRAY_CONCAT_AGG(spender_list) AS all_spender
  FROM daily_revenue
  GROUP BY revenue_date, launch_phase
),

-- Step 3: Calculate unique paying users and per-user revenue
revenue_metrics AS (
  SELECT
    revenue_date,
    launch_phase,
    iap_revenue,
    ad_revenue,
    ARRAY_LENGTH(ARRAY(SELECT DISTINCT u FROM UNNEST(all_spender) AS u)) AS total_spender,
    SAFE_DIVIDE(iap_revenue, ARRAY_LENGTH(ARRAY(SELECT DISTINCT u FROM UNNEST(all_spender) AS u))) AS iap_per_spender,
    SAFE_DIVIDE(ad_revenue, ARRAY_LENGTH(ARRAY(SELECT DISTINCT u FROM UNNEST(all_spender) AS u))) AS ad_per_spender,
    SAFE_DIVIDE(iap_revenue + ad_revenue, ARRAY_LENGTH(ARRAY(SELECT DISTINCT u FROM UNNEST(all_spender) AS u))) AS total_per_spender
  FROM pivot_revenue
)

SELECT a.*, DAU
FROM revenue_metrics a
full join (select a.date, count (distinct user_id) as DAU from `tactile-external.data_analyst_test.activity` a group by 1 ) b
on a.revenue_date = b.date
WHERE launch_phase != 'Other'
ORDER BY launch_phase, revenue_date;


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df = monetisation
# Convert date column to datetime
df['revenue_date'] = pd.to_datetime(df['revenue_date'])

# Filter only US - Official Launch
us_df = df[df['launch_phase'] == 'US - Official Launch'].sort_values('revenue_date')

# Create subplots: IAP and Ads revenue
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Daily In-App Revenue (US)", "Daily Ads Revenue (US)"),
    shared_xaxes=True
)

# --- Plot 1: In-App Revenue ---
fig.add_trace(
    go.Scatter(
        x=us_df['revenue_date'],
        y=us_df['iap_revenue'],
        mode='lines+markers',
        name='IAP Revenue',
        line=dict(color='blue')
    ),
    row=1, col=1
)

# --- Plot 2: Ads Revenue ---
fig.add_trace(
    go.Scatter(
        x=us_df['revenue_date'],
        y=us_df['ad_revenue'],
        mode='lines+markers',
        name='Ads Revenue',
        line=dict(color='red')
    ),
    row=2, col=1
)

# Layout updates
fig.update_layout(
    height=700,
    width=1000,
    title_text="Daily Revenue for US - Official Launch",
    showlegend=True
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=2, col=1)

fig.show()
